# Import necessary packages

In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from random import sample

import re

from sklearn.ensemble import RandomForestRegressor

from sklearn.model_selection import TimeSeriesSplit, RandomizedSearchCV

from timeit import default_timer as timer

from sklearn.metrics import mean_absolute_error, mean_squared_error

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Import original data and preprocessed data

In [3]:
sales_train_val = pd.read_csv('/content/drive/MyDrive/Colab Notebooks_Weekly Prediction/sales_train_validation.csv')
calendar = pd.read_csv('/content/drive/MyDrive/Colab Notebooks_Weekly Prediction/calendar.csv')

**Note: This Step is for the extraction of California State and Foods category**

In [4]:
sales_train_val = sales_train_val[(sales_train_val['state_id'] == 'CA') & (sales_train_val['cat_id'] == 'FOODS')]

In [5]:
sales_train_val

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
1612,FOODS_1_001_CA_1_validation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,3,0,0,1,...,0,2,0,4,1,1,0,1,1,0
1613,FOODS_1_002_CA_1_validation,FOODS_1_002,FOODS_1,FOODS,CA_1,CA,0,1,0,1,...,1,3,1,0,0,1,2,0,0,0
1614,FOODS_1_003_CA_1_validation,FOODS_1_003,FOODS_1,FOODS,CA_1,CA,0,0,0,0,...,3,0,2,1,1,0,1,0,1,0
1615,FOODS_1_004_CA_1_validation,FOODS_1_004,FOODS_1,FOODS,CA_1,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1616,FOODS_1_005_CA_1_validation,FOODS_1_005,FOODS_1,FOODS,CA_1,CA,3,9,3,3,...,3,1,1,2,0,2,2,1,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12191,FOODS_3_823_CA_4_validation,FOODS_3_823,FOODS_3,FOODS,CA_4,CA,1,1,1,4,...,3,1,2,2,0,1,1,3,3,0
12192,FOODS_3_824_CA_4_validation,FOODS_3_824,FOODS_3,FOODS,CA_4,CA,0,0,0,1,...,0,1,1,1,0,2,0,0,0,1
12193,FOODS_3_825_CA_4_validation,FOODS_3_825,FOODS_3,FOODS,CA_4,CA,2,3,2,1,...,0,2,0,1,3,1,1,0,2,0
12194,FOODS_3_826_CA_4_validation,FOODS_3_826,FOODS_3,FOODS,CA_4,CA,0,0,0,0,...,1,2,2,1,4,5,1,7,0,4


In [6]:
INPUT_DIR_2 = r'/content/drive/MyDrive/Colab Notebooks_Weekly Prediction/List of Product ID according to 4 demand patterns/California/Foods'

list_intermittent = pd.read_csv(f'{INPUT_DIR_2}/Intermittent_ID.csv')
list_lumpy = pd.read_csv(f'{INPUT_DIR_2}/Lumpy_ID.csv')
list_erratic = pd.read_csv(f'{INPUT_DIR_2}/Erratic_ID.csv')
list_smooth = pd.read_csv(f'{INPUT_DIR_2}/Smooth_ID.csv')

list_intermittent = list_intermittent['0'].values.tolist()
list_lumpy = list_lumpy['0'].values.tolist()
list_erratic = list_erratic['0'].values.tolist()
list_smooth = list_smooth['0'].values.tolist()

sales_intermittent = sales_train_val[sales_train_val.id.isin(list_intermittent)]
sales_lumpy = sales_train_val[sales_train_val.id.isin(list_lumpy)]
sales_erratic = sales_train_val[sales_train_val.id.isin(list_erratic)]
sales_smooth = sales_train_val[sales_train_val.id.isin(list_smooth)]

In [7]:
len(sales_intermittent)

1301

In [8]:
len(sales_lumpy)

395

In [9]:
len(sales_erratic)

458

In [10]:
len(sales_smooth)

3594

**Check by looking at column "id" to verify if we choose the right product (FOODS or HOBBIES or HOUSEHOLD)**

In [11]:
sales_erratic

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
1665,FOODS_1_055_CA_1_validation,FOODS_1_055,FOODS_1,FOODS,CA_1,CA,2,5,2,5,...,0,0,1,2,3,12,5,1,1,6
1669,FOODS_1_059_CA_1_validation,FOODS_1_059,FOODS_1,FOODS,CA_1,CA,0,0,0,0,...,0,0,0,0,0,0,2,0,0,0
1678,FOODS_1_068_CA_1_validation,FOODS_1_068,FOODS_1,FOODS,CA_1,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1692,FOODS_1_082_CA_1_validation,FOODS_1_082,FOODS_1,FOODS,CA_1,CA,0,0,0,0,...,4,14,8,6,9,10,10,12,8,7
1712,FOODS_1_103_CA_1_validation,FOODS_1_103,FOODS_1,FOODS,CA_1,CA,7,1,1,2,...,1,0,1,2,1,1,3,3,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12145,FOODS_3_777_CA_4_validation,FOODS_3_777,FOODS_3,FOODS,CA_4,CA,0,0,0,0,...,4,1,1,0,1,0,0,0,0,0
12169,FOODS_3_801_CA_4_validation,FOODS_3_801,FOODS_3,FOODS,CA_4,CA,0,0,0,0,...,0,3,2,2,0,0,0,1,2,3
12177,FOODS_3_809_CA_4_validation,FOODS_3_809,FOODS_3,FOODS,CA_4,CA,0,0,0,0,...,1,1,1,1,0,7,1,2,4,0
12179,FOODS_3_811_CA_4_validation,FOODS_3_811,FOODS_3,FOODS,CA_4,CA,0,0,0,0,...,2,7,4,1,0,0,2,3,5,2


# User-defined functions to calculate Metrics

In [12]:
ROUNDING_DECIMAL = 4

def mase_calculation(ts, prediction):
    divisor = 0
    for i in range(1, ts.shape[0]):
        divisor = divisor + abs(ts.iloc[i] - ts.iloc[i-1])
    divisor = divisor/(ts.shape[0] - 1)
    diff    = abs(ts - prediction[:ts.shape[0]])/divisor
    mase    = diff.mean()
    return mase

def mape_calculation(actual, pred): 
    if not all([isinstance(actual, np.ndarray), isinstance(pred, np.ndarray)]):
        actual, pred = np.array(actual), np.array(pred)
    mask = (actual != 0)
    return round((np.fabs(actual - pred)/actual)[mask].mean()*100, ROUNDING_DECIMAL)

def wmape_calculation(actual, pred):
    if not all([isinstance(actual, np.ndarray), isinstance(pred, np.ndarray)]):
        actual, pred = np.array(actual), np.array(pred)
    return round((np.sum(np.absolute(actual-pred))/np.sum(actual))*100, ROUNDING_DECIMAL)

def smape_calculation(actual, predicted):
    if not all([isinstance(actual, np.ndarray), isinstance(predicted, np.ndarray)]):
        actual, predicted = np.array(actual), np.array(predicted)
    return round(np.mean(np.abs(predicted - actual) / ((np.abs(predicted) + np.abs(actual))/2))*100, ROUNDING_DECIMAL)

# Format sales data and then merge with calendar data

In [13]:
sales_pattern = sales_erratic.copy()

In [14]:
sales_pattern_py = sales_pattern.copy()

list_pattern_py = sales_pattern_py.id.unique().tolist()
list_pattern_py = [(re.sub(r'_validation', r"", x)) for x in list_pattern_py]

sales_pattern_py = sales_pattern_py.drop(['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], axis=1)
df_pattern = sales_pattern_py.melt(['id'], var_name='Date').pivot(index = ['Date'], columns = 'id', values = 'value').reset_index()
df_pattern.drop('Date', axis=1, inplace=True)
df_pattern.index = pd.date_range('2011-01-29', periods=df_pattern.shape[0], freq="D")
df_pattern.index.names = ['Date']
df_pattern = df_pattern.astype('float64')

df_pattern.iloc[:, 1:].columns = df_pattern.iloc[:, 1:].columns.str.rstrip('_validation')
df_pattern.columns = df_pattern.columns.str.replace(r'_validation', '')
df_pattern.reset_index(inplace=True)

calendar['Date'] = pd.to_datetime(calendar['date'])

dataframe = pd.merge(df_pattern,
                     calendar[['wm_yr_wk', 'Date']],
                     on = 'Date')

not_enough_seven_days = dataframe.groupby('wm_yr_wk').agg({'Date': 'count'})[dataframe.groupby('wm_yr_wk').agg({'Date': 'count'})['Date'] < 7].index.tolist()
dataframe = dataframe[~dataframe['wm_yr_wk'].isin(not_enough_seven_days)]

dataframe.set_index('wm_yr_wk', inplace=True)
dataframe.drop('Date', axis=1, inplace=True)

dataframe_weekly = dataframe.groupby('wm_yr_wk').sum().reset_index()

In [15]:
dataframe_weekly

,wm_yr_wk,FOODS_1_003_CA_4,FOODS_1_004_CA_2,FOODS_1_019_CA_3,FOODS_1_022_CA_3,FOODS_1_025_CA_4,FOODS_1_027_CA_3,FOODS_1_030_CA_3,FOODS_1_032_CA_3,FOODS_1_034_CA_2,...,FOODS_3_809_CA_4,FOODS_3_811_CA_2,FOODS_3_811_CA_4,FOODS_3_815_CA_3,FOODS_3_816_CA_3,FOODS_3_816_CA_4,FOODS_3_822_CA_2,FOODS_3_822_CA_3,FOODS_3_825_CA_3,FOODS_3_827_CA_3
0,11101,2.0,15.0,30.0,3.0,5.0,12.0,3.0,28.0,1.0,...,0.0,18.0,19.0,0.0,31.0,7.0,22.0,29.0,0.0,0.0
1,11102,7.0,31.0,9.0,10.0,4.0,23.0,1.0,18.0,0.0,...,9.0,31.0,29.0,0.0,55.0,7.0,14.0,36.0,3.0,0.0
2,11103,6.0,28.0,0.0,11.0,6.0,14.0,3.0,17.0,0.0,...,3.0,25.0,31.0,0.0,117.0,3.0,27.0,50.0,7.0,0.0
3,11104,3.0,37.0,2.0,3.0,4.0,31.0,4.0,20.0,1.0,...,9.0,15.0,19.0,0.0,49.0,10.0,16.0,37.0,3.0,0.0
4,11105,7.0,40.0,0.0,8.0,4.0,24.0,3.0,17.0,2.0,...,3.0,0.0,0.0,1.0,61.0,5.0,17.0,34.0,2.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
268,11608,2.0,39.0,50.0,3.0,6.0,17.0,2.0,23.0,3.0,...,11.0,22.0,19.0,0.0,64.0,14.0,11.0,37.0,19.0,0.0
269,11609,23.0,47.0,3.0,4.0,2.0,18.0,0.0,21.0,2.0,...,6.0,34.0,22.0,0.0,83.0,10.0,14.0,39.0,5.0,0.0
270,11610,2.0,56.0,6.0,4.0,4.0,22.0,1.0,22.0,0.0,...,4.0,32.0,14.0,0.0,22.0,19.0,28.0,52.0,0.0,0.0
271,11611,2.0,31.0,11.0,10.0,5.0,14.0,2.0,43.0,1.0,...,4.0,34.0,15.0,0.0,59.0,3.0,18.0,37.0,3.0,0.0


In [16]:
lags = list(range(1, 25))
moving_averages = list(range(1, 25)) 
stds = list(range(2, 25))
maximums = list(range(1, 25))
minimums = list(range(1, 25))
totals = list(range(1, 25))
num_days_zeros = list(range(1, 25))

correlation_threshold = 0.8

end_train_week = 11610

num_id_sample_for_tuning = 200

In [17]:
sample_list = sample(list_pattern_py, num_id_sample_for_tuning)

In [18]:
dataframe_check = dataframe_weekly[sample_list].iloc[-2:].T

dataframe_check['Check'] = np.where((dataframe_check[271] == 0) &
                                    (dataframe_check[272] == 0), 1, 0)

sample_removed = dataframe_check[dataframe_check['Check'] == 1].index.tolist()

In [19]:
sample_list_filtered = [x for x in sample_list if (x not in sample_removed)]

In [20]:
len(sample_list_filtered)

157

In [21]:
start = timer()

df_output_hyper_params = pd.DataFrame()
    
for product in sample_list_filtered:
    print('Currently Running: %s' % product)
    print('Progressing: {0} %'.format(round(sample_list_filtered.index(product) / len(sample_list_filtered) * 100, 2)))

    dataframe_product = dataframe_weekly[['wm_yr_wk', product]]

    # Create Lag variables
    dataframe_product = dataframe_product.assign(**{
        f'{product} (t-{lag})': dataframe_product[product].shift(lag)
        for lag in lags
    })

    # Create Rolling Moving Average variables
    dataframe_product = dataframe_product.assign(**{
        f'{product} (mva_{moving_average})': dataframe_product[product].shift(1).rolling(moving_average).mean()
        for moving_average in moving_averages
    })

    # Create Rolling Standard Deviation variables
    dataframe_product = dataframe_product.assign(**{
        f'{product} (std_{std})': dataframe_product[product].shift(1).rolling(std).std()
        for std in stds
    })

    # Create Rolling Maximum variables
    dataframe_product = dataframe_product.assign(**{
        f'{product} (max_{maximum})': dataframe_product[product].shift(1).rolling(maximum).max()
        for maximum in maximums
    })

    # Create Rolling Minimum variables
    dataframe_product = dataframe_product.assign(**{
        f'{product} (min_{minimum})': dataframe_product[product].shift(1).rolling(minimum).min()
        for minimum in minimums
    })

    # Create Rolling Total variables 
    dataframe_product = dataframe_product.assign(**{
        f'{product} (total_{total})': dataframe_product[product].shift(1).rolling(total).sum()
        for total in totals
    })


    dataframe_product.dropna(inplace=True)
    dataframe_product.set_index('wm_yr_wk', inplace=True)

    train_data = dataframe_product[dataframe_product.index <= end_train_week]

    corr_matrix = train_data.loc[:, train_data.columns != product].corr().abs()
    high_corr_var=np.where(corr_matrix > correlation_threshold)
    high_corr_var=[(corr_matrix.columns[x],corr_matrix.columns[y]) for x,y in zip(*high_corr_var) if x!=y and x<y]

    train_data.drop([i[1] for i in high_corr_var], axis=1, inplace=True)

    X_train = train_data.drop(product, axis=1)
    y_train = train_data[product]  

    tscv = TimeSeriesSplit(n_splits=3)

    params = {'n_estimators': [30, 50],
              'max_depth': list(range(3,11)),
              'min_samples_leaf': range(4,11),
              'max_features': ['sqrt', 'log2', None],
              'bootstrap': [True], #We force bootstrap
              'max_samples': [0.6, 0.7, 0.8, 0.9]}

    forest = RandomForestRegressor(n_jobs=1)

    regressor = RandomizedSearchCV(estimator=forest,
                                  param_distributions=params,
                                  cv = tscv, 
                                  scoring='neg_mean_absolute_percentage_error',
                                  n_iter=150,
                                  verbose=1)
    regressor.fit(X_train, y_train)
    best_params_temp = regressor.best_params_

    df_output_temp = pd.DataFrame.from_dict(best_params_temp, orient='index').T
    df_output_temp['Product'] = product

    df_output_hyper_params = df_output_hyper_params.append(df_output_temp, ignore_index=True)

    
end = timer()
print('This line of code took {} minutes'.format((end-start) / 60))

Currently Running: FOODS_3_514_CA_3
Progressing: 0.0 %
Fitting 3 folds for each of 150 candidates, totalling 450 fits
Currently Running: FOODS_3_582_CA_4
Progressing: 0.64 %
Fitting 3 folds for each of 150 candidates, totalling 450 fits
Currently Running: FOODS_3_069_CA_3
Progressing: 1.27 %
Fitting 3 folds for each of 150 candidates, totalling 450 fits
Currently Running: FOODS_1_116_CA_3
Progressing: 1.91 %
Fitting 3 folds for each of 150 candidates, totalling 450 fits
Currently Running: FOODS_3_629_CA_4
Progressing: 2.55 %
Fitting 3 folds for each of 150 candidates, totalling 450 fits
Currently Running: FOODS_2_042_CA_3
Progressing: 3.18 %
Fitting 3 folds for each of 150 candidates, totalling 450 fits
Currently Running: FOODS_3_607_CA_3
Progressing: 3.82 %
Fitting 3 folds for each of 150 candidates, totalling 450 fits
Currently Running: FOODS_3_150_CA_3
Progressing: 4.46 %
Fitting 3 folds for each of 150 candidates, totalling 450 fits
Currently Running: FOODS_3_656_CA_3
Progressing: 

In [22]:
df_output_hyper_params.to_csv('/content/drive/MyDrive/Colab Notebooks_Weekly Prediction/Random Forest (Completed Pipeline)/Foods/Erratic_Params_Tuning_Random Forest.csv')

# Decide optimum set of parameters for Random Forest algorithm

In [23]:
# See if any values larger than 1 in column "Product"
# If yes, the corresponding set of parameters occur more than once and those sets should be considered first
df_output_hyper_params.groupby(['n_estimators', 'max_depth', 'min_samples_leaf', 'max_features', 'max_samples'])[['Product']].count().sort_values(by = 'Product', ascending=False)

Product
n_estimators max_depth min_samples_leaf max_features max_samples         
30           3         4                sqrt         0.7                1
             4         4                sqrt         0.6                1
50           8         6                sqrt         0.6                1
             6         4                sqrt         0.6                1
             3         4                sqrt         0.6                1
                                        log2         0.6                1
30           10        7                sqrt         0.9                1
                                                     0.8                1
                       4                sqrt         0.7                1
                                        log2         0.7                1
             9         7                log2         0.9                1
                       5                sqrt         0.9                1
                                                     0.8                1
             8         10               sqrt         0.7                1
                       6                sqrt         0.6                1
                       5                log2         0.6                1
                       4                log2         0.7                1
             6         10               sqrt         0.6                1
                       8                log2         0.6                1
                       7                log2         0.9                1
                                                     0.6                1
             5         9                sqrt         0.7                1
                                                     0.6                1
                       8                sqrt         0.6                1
                       5                sqrt         0.9                1
                                        log2         0.9                1
             4         8                sqrt         0.8                1
50           10        8                sqrt         0.9                1

In [24]:
df_output_hyper_params.groupby(['n_estimators'])[['Product']].count()

,Product
n_estimators,
30,108
50,49


In [25]:
df_output_hyper_params.groupby(['max_depth'])[['Product']].count()

,Product
max_depth,
3,14
4,19
5,24
6,23
7,14
8,24
9,18
10,21


In [26]:
# See what value of parameter "max_depth" occurs the most frequently
df_output_hyper_params.groupby(['min_samples_leaf'])[['Product']].count()

,Product
min_samples_leaf,
4,39
5,25
6,16
7,17
8,22
9,16
10,22


In [27]:
# See what value of parameter "learning_rate" occurs the most frequently
df_output_hyper_params.groupby(['max_features'])[['Product']].count()

,Product
max_features,
log2,9
sqrt,19


In [28]:
# See what value of parameter "colsample_bytree" occurs the most frequently
df_output_hyper_params.groupby(['max_samples'])[['Product']].count()

,Product
max_samples,
0.6,38
0.7,32
0.8,37
0.9,50


# Implement Random Forest on Test Set with optimum set of parameters

### Specify optimum set of parameters

In [29]:
n_estimators_chosen = 30
max_depth_chosen = 6
min_samples_leaf_chosen = 4
max_features_chosen = 'sqrt'
max_samples_chosen = 0.9

In [30]:
len(list_pattern_py)

458

In [31]:
list_pattern_temp = list_pattern_py.copy()

In [32]:
start = timer()

df_output_rf_pattern = pd.DataFrame()
for product in list_pattern_temp:
  print('Currently Running: %s' % product)
  print('Progressing: {0} %'.format(round(list_pattern_temp.index(product) / len(list_pattern_temp) * 100, 2)))
  dataframe_product = dataframe_weekly[['wm_yr_wk', product]]
  # Create Lag variables
  dataframe_product = dataframe_product.assign(**{
      f'{product} (t-{lag})': dataframe_product[product].shift(lag)
      for lag in lags
  })

  # Create Rolling Moving Average variables
  dataframe_product = dataframe_product.assign(**{
      f'{product} (mva_{moving_average})': dataframe_product[product].shift(1).rolling(moving_average).mean()
      for moving_average in moving_averages
  })

  # Create Rolling Standard Deviation variables
  dataframe_product = dataframe_product.assign(**{
      f'{product} (std_{std})': dataframe_product[product].shift(1).rolling(std).std()
      for std in stds
  })

  # Create Rolling Maximum variables
  dataframe_product = dataframe_product.assign(**{
      f'{product} (max_{maximum})': dataframe_product[product].shift(1).rolling(maximum).max()
      for maximum in maximums
  })

  # Create Rolling Minimum variables
  dataframe_product = dataframe_product.assign(**{
      f'{product} (min_{minimum})': dataframe_product[product].shift(1).rolling(minimum).min()
      for minimum in minimums
  })

  # Create Rolling Total variables 
  dataframe_product = dataframe_product.assign(**{
      f'{product} (total_{total})': dataframe_product[product].shift(1).rolling(total).sum()
      for total in totals
  })

  dataframe_product.dropna(inplace=True)
  dataframe_product.set_index('wm_yr_wk', inplace=True)

  train_data = dataframe_product[dataframe_product.index <= end_train_week]
  test_data = dataframe_product[dataframe_product.index > end_train_week]

  corr_matrix = train_data.loc[:, train_data.columns != product].corr().abs()
  high_corr_var=np.where(corr_matrix > correlation_threshold)
  high_corr_var=[(corr_matrix.columns[x],corr_matrix.columns[y]) for x,y in zip(*high_corr_var) if x!=y and x<y]

  train_data.drop([i[1] for i in high_corr_var], axis=1, inplace=True)
  test_data.drop([i[1] for i in high_corr_var], axis=1, inplace=True)

  X_train = train_data.drop(product, axis=1)
  y_train = train_data[product]

  X_test = test_data.drop(product, axis=1)
  y_test = test_data[product]

  forest = RandomForestRegressor(n_jobs=1,
                                n_estimators = n_estimators_chosen,
                                min_samples_leaf = min_samples_leaf_chosen,
                                max_samples = max_samples_chosen,
                                max_features = max_features_chosen,
                                max_depth = max_depth_chosen,
                                bootstrap = [True],
                                random_state = 0)
  

  forest.fit(X_train,y_train)
  y_pred = forest.predict(X_test)
  df_output_temp = pd.DataFrame({'Actual Data': y_test, 
                                  'Forecast': y_pred, 
                                  'Product': [product for num_rows in range(len(y_pred))]
                                  })
  
  df_output_rf_pattern = df_output_rf_pattern.append(df_output_temp, ignore_index=False)  

end = timer()
print('This line of code took {} minutes'.format((end-start) / 60))

Currently Running: FOODS_1_055_CA_1
Progressing: 0.0 %
Currently Running: FOODS_1_059_CA_1
Progressing: 0.22 %
Currently Running: FOODS_1_068_CA_1
Progressing: 0.44 %
Currently Running: FOODS_1_082_CA_1
Progressing: 0.66 %
Currently Running: FOODS_1_103_CA_1
Progressing: 0.87 %
Currently Running: FOODS_1_113_CA_1
Progressing: 1.09 %
Currently Running: FOODS_1_115_CA_1
Progressing: 1.31 %
Currently Running: FOODS_1_128_CA_1
Progressing: 1.53 %
Currently Running: FOODS_1_131_CA_1
Progressing: 1.75 %
Currently Running: FOODS_1_132_CA_1
Progressing: 1.97 %
Currently Running: FOODS_1_163_CA_1
Progressing: 2.18 %
Currently Running: FOODS_1_171_CA_1
Progressing: 2.4 %
Currently Running: FOODS_1_179_CA_1
Progressing: 2.62 %
Currently Running: FOODS_1_180_CA_1
Progressing: 2.84 %
Currently Running: FOODS_1_216_CA_1
Progressing: 3.06 %
Currently Running: FOODS_2_003_CA_1
Progressing: 3.28 %
Currently Running: FOODS_2_027_CA_1
Progressing: 3.49 %
Currently Running: FOODS_2_058_CA_1
Progressing: 3

In [33]:
df_output_rf_pattern

,Actual Data,Forecast,Product
wm_yr_wk,,,
11611,21.0,16.141787,FOODS_1_055_CA_1
11612,24.0,15.625939,FOODS_1_055_CA_1
11611,0.0,3.934715,FOODS_1_059_CA_1
11612,0.0,2.078299,FOODS_1_059_CA_1
11611,1.0,2.285160,FOODS_1_068_CA_1
...,...,...,...
11612,7.0,8.633077,FOODS_3_809_CA_4
11611,15.0,18.312699,FOODS_3_811_CA_4
11612,32.0,16.194457,FOODS_3_811_CA_4


In [34]:
df_output_rf_pattern.to_csv('Erratic_Random_Forest_Forecast_Values.csv')

In [35]:
# User-defined funcion to calculate metrics

def get_metrics_result_all_params(data):
    metrics_df = {}
    metrics_df['MASE'] = mase_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['WMAPE'] = wmape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['SMAPE'] = smape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['MAPE'] = mape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['MAE'] = mean_absolute_error(data['Actual Data'], data['Forecast'])
    metrics_df['RMSE'] = np.sqrt(mean_squared_error(data['Actual Data'], data['Forecast']))
    return pd.Series(metrics_df)

In [36]:
# Start calculating metrics 
start = timer()

df_forecast_metrics = df_output_rf_pattern.groupby('Product').apply(get_metrics_result_all_params).reset_index()

end = timer()
print('This line of code took {} minutes'.format((end-start) / 60))

This line of code took 0.022153042783338607 minutes


In [37]:
df_forecast_metrics

,Product,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
0,FOODS_1_003_CA_4,0.496686,63.2146,71.3759,83.2936,3.476804,3.671550
1,FOODS_1_004_CA_2,0.689873,31.0443,29.3771,37.7767,12.417711,15.087273
2,FOODS_1_019_CA_3,3.170868,634.1736,161.4631,318.9837,34.879547,34.880172
3,FOODS_1_022_CA_3,0.510144,102.0289,162.4684,76.8991,5.101445,5.720569
4,FOODS_1_025_CA_4,inf,27.3756,21.8877,27.3756,1.368781,1.865212
...,...,...,...,...,...,...,...
453,FOODS_3_816_CA_4,0.598010,81.8330,93.0521,126.5598,7.774133,8.031336
454,FOODS_3_822_CA_2,0.581444,38.7629,37.9158,52.3578,5.232998,6.107832
455,FOODS_3_822_CA_3,0.646155,19.3104,19.9488,17.6977,8.400009,10.233471
456,FOODS_3_825_CA_3,0.823597,74.8724,63.5767,120.7336,4.117984,4.833532


In [40]:
df_forecast_metrics.to_csv('Random_Forest_Erratic_Forecast_Metrics.csv')